# Transformer

- Encoder: O Encoder recebe a sequência de entrada (por exemplo, uma frase em português) e a processa para criar uma representação vetorial de alta qualidade dessa sequência. Essa representação captura o significado e o contexto de cada palavra em relação às outras palavras da frase.
- Decoder: O Decoder recebe a representação gerada pelo Encoder e a utiliza para gerar uma sequência de saída (por exemplo, a tradução da frase para o inglês).


# Masked Language Model

Um Masked Language Model (MLM) é um tipo de modelo de linguagem amplamente utilizado em processamento de linguagem natural.

Durante o treinamento, uma parte dos tokens (palavras ou subpalavras) no texto de entrada é substituída por um token especial de máscara, como "[MASK]". O objetivo do modelo é prever corretamente quais eram os tokens originais que foram mascarados.

Essa estratégia obriga o modelo a aprender contextos ricos e relações entre as palavras, o que é fundamental para o desempenho em diversas tarefas, como análise de sentimentos, tradução, e resposta a perguntas. Modelos famosos que utilizam essa técnica incluem o BERT, que demonstrou ganhos significativos em várias benchmarks de NLP .


## IMBD

Para este projeto, será utilizado a base do IMBD.

## Configuração

In [1]:
import os

os.environ[ "KERAS_BACKEND" ] = "torch"  # or jax, or tensorflow

import keras_hub

import keras
from keras import layers
from keras.layers import TextVectorization

from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint

/Users/viniciuscandeia/development/transformer_mlm/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
@dataclass
class Config:
    MAX_LEN = 256
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128
    NUM_HEAD = 8  # used in bert model
    FF_DIM = 128  # used in bert model
    NUM_LAYERS = 1


config = Config()

# Carregando os dados

Primeiro, vamos carregar os dados que estão na pasta "aclImbd".

Duas funções serão utilizadas para isso:

- Uma irá criar uma lista contendo o conteúdo dos arquivos.
- A outra ficará responsável por criar um dataframe.

In [3]:
def get_text_list_from_files( files ) -> list[ str ]:
    """
       Esta função irá retornar uma lista contendo todas as frases dos arquivos.
    """
    text_list: list[ str ] = [ ]
    for name in files:
        with open( name ) as f:
            for line in f:
                text_list.append( line )
    return text_list


def get_data_from_text_files( folder_name ):
    # Arquivos de texto com avaliações positivas e negativas
    pos_files = glob.glob( f"aclImdb/{folder_name}/pos/*.txt" )
    neg_files = glob.glob( f"aclImdb/{folder_name}/neg/*.txt" )

    # Listas com as avaliações
    pos_texts: list[ str ] = get_text_list_from_files( pos_files )
    neg_texts: list[ str ] = get_text_list_from_files( neg_files )

    # Criação de um dataframe, com colunas chamadas "review" e "sentiment"
    df = pd.DataFrame(
            {
                # Concatenação das listas
                "review": pos_texts + neg_texts,
                # Criação de uma nova lista
                "sentiment": [ 0 ] * len( pos_texts ) + [ 1 ] * len( neg_texts ),
            }
    )

    # Sample -> pega uma amostra aleatória
    # len(df) -> do tamanho do df original
    # reset_index -> ao usar sample, o índice original das linhas é mantido
    df = df.sample( len( df ) ).reset_index( drop = True )
    return df


train_df = get_data_from_text_files( "train" )
test_df = get_data_from_text_files( "test" )

# Concatenação dos dataframes para realizar pré-processamento em toda a base
all_data = pd.concat( [ train_df, test_df ], ignore_index = True )

In [4]:
train_df.head()

,review,sentiment
0,This movie is a classic in every sense of the ...,0
1,Even if you could get past the idea that these...,1
2,First let me preface this post by saying that ...,1
3,"""The Incredible Melting Man"" is a fantasticall...",0
4,this movie was clearly done poorly and in a ru...,1


In [5]:
import tensorflow as tf


def custom_standardization( input_data ):
    """
        Normalização de texto.
    """
    # Converter todas as letras para minúsculas
    lowercase = tf.strings.lower( input_data )

    # Expressão Regular para remover a tag HTML
    stripped_html = tf.strings.regex_replace( lowercase, "<br />", " " )

    # Remover caracteres especiais
    return tf.strings.regex_replace(
            stripped_html,
            "[%s]" % re.escape( "!#$%&'()*+,-./:;<=>?@\^_`{|}~" ),
            ""
    )

## Vetorização de Texto

Para um transformer, a vetorização de um texto é o processo fundamental de transformar o texto bruto em uma representação numérica que o modelo possa entender e processar. Em essência, é como traduzir a linguagem humana para a linguagem matemática que o transformer consegue trabalhar.

Imagine que o transformer é um computador que só entende números. Para que ele consiga ler e compreender um texto, precisamos converter cada palavra (ou parte da palavra) em um conjunto de números. Esse conjunto de números é o que chamamos de vetor.

Aqui está um detalhamento do processo de vetorização para um transformer:

1. Tokenização: O primeiro passo é dividir o texto em unidades menores, chamadas tokens. Um token pode ser uma palavra inteira, parte de uma palavra (subpalavra), ou até mesmo um caractere. Por exemplo, a frase "O gato comeu o rato" poderia ser tokenizada como: ["O", "gato", "comeu", "o", "rato"].

2. Criação do Vocabulário: Em seguida, é criado um vocabulário, que é uma lista de todos os tokens únicos presentes no conjunto de dados de treinamento do modelo. Cada token nesse vocabulário recebe um índice único.

3. Indexação: Cada token no texto de entrada é então mapeado para o seu índice correspondente no vocabulário. Usando o exemplo anterior e supondo um vocabulário, os tokens poderiam ser convertidos em índices como: [10, 25, 50, 10, 75].

4. Embedding: A etapa crucial para transformers é a criação de embeddings. Em vez de simplesmente usar os índices brutos, cada índice é transformado em um vetor denso de números reais. Esse vetor captura o significado semântico e as relações entre as palavras.

    - Word Embeddings: Cada palavra (ou token) é associada a um vetor de baixa dimensionalidade (por exemplo, 512 ou 768 dimensões). Palavras com significados semelhantes tendem a ter vetores próximos no espaço vetorial. Por exemplo, os vetores para "gato" e "felino" provavelmente estarão mais próximos do que os vetores para "gato" e "carro".

    - Positional Embeddings: Transformers também precisam entender a ordem das palavras em uma frase. Para isso, são adicionados embeddings posicionais aos word embeddings. Esses vetores codificam a posição de cada token na sequência, permitindo que o modelo diferencie entre "o gato comeu o rato" e "o rato comeu o gato".

5. Input para o Transformer: Os vetores resultantes (a soma dos word embeddings e positional embeddings para cada token) são então alimentados como entrada para as diferentes camadas do transformer (como as camadas de atenção).

In [6]:
def get_vectorize_layer( texts: list[ str ], vocab_size: int, max_seq: int, special_tokens: list = [ "[MASK]" ] ):
    """Build Text vectorization layer

    Args:
      texts (list): List of string i.e input texts
      vocab_size (int): vocab size
      max_seq (int): Maximum sequence length.
      special_tokens (list, optional): List of special tokens. Defaults to ['[MASK]'].

    Returns:
        layers.Layer: Return TextVectorization Keras Layer
    """
    vectorize_layer = TextVectorization(
            max_tokens = vocab_size,
            output_mode = "int",
            standardize = custom_standardization,
            output_sequence_length = max_seq,
    )
    vectorize_layer.adapt( texts )

    # Insert mask token in vocabulary
    vocab = vectorize_layer.get_vocabulary()
    vocab = vocab[ 2: vocab_size - len( special_tokens ) ] + [ "[mask]" ]
    vectorize_layer.set_vocabulary( vocab )
    return vectorize_layer